# 지주, 금융, 중국 종목 (제외 종목) 목록 만들기

In [3]:
import FinanceDataReader as fdr
import pandas as pd
import pickle

In [4]:
df_kospi = fdr.StockListing('KOSPI')
df_kosdaq = fdr.StockListing('KOSDAQ')
# df_del = fdr.StockListing('KRX-DELISTING')
df = pd.concat([df_kospi,df_kosdaq],axis=0)

#  Sector에서 기타금융업, Industry에서 지주가 들어가는 회사. 중국회사.
df_fin = df[df.Sector=='기타 금융업'].loc[:,['Symbol','Name']]
df_fin['Category'] = '금융'
df_holding = df.loc[df['Industry'].str.contains('지주',na=False)].loc[:,['Symbol','Name']]
df_holding['Category'] = '지주'
df_china = df.loc[df['Symbol'].str.contains('^9', regex=True)].loc[:,['Symbol','Name']]
df_china['Category'] = '중국'
df_exclu = pd.concat([df_fin,df_holding,df_china],axis=0)
df_exclu.rename(columns={'Symbol':'StockCode'},inplace=True)
with open('Raw_Finance/listExclusive.pickle', 'wb') as handle:
    pickle.dump(df_exclu, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 가격자료 (OHLCV) 별도로 저장하기

In [ ]:
listStock = case.change.columns
dateStart = case.change.index[0]
dateEnd = case.change.index[-1]
priceData = {}
for stockCode in listStock:
    stockData = fdr.DataReader(stockCode,start=dateStart,end=dateEnd)
    priceData[stockCode] = stockData
    
with open('Raw_Price/OHLCV_all.pickle', 'wb') as handle:
    pickle.dump(priceData, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# 거래량 거래대금 저장.

listStock = case.change.columns
dateStart = case.change.index[0]
dateEnd = case.change.index[-1]
tradingVolumeData = {}
tradingCostData = {}
for stockCode in listStock:
    stockData = stock.get_market_ohlcv_by_date(dateStart, dateEnd, stockCode,adjusted=False)
    
    tradingVolumeData[stockCode] = stockData['거래량']
    tradingCostData[stockCode] = stockData['거래대금']
    time.sleep(1)

with open('Raw_Price/tradingVolume_all.pickle', 'wb') as handle:
    pickle.dump(tradingVolumeData, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('Raw_Price/tradingCost_all.pickle', 'wb') as handle:
    pickle.dump(tradingCostData, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 네이버 증권 자료 다운받기

# dart 자료와 네이버 증권 자료 병합하기

In [ ]:
# dart 자료에서 특정 index삭제하여 2021 분기 자료 삭제
f=case.finance
f.reset_index(drop=True,inplace=True)
indexName = f[f['period']==202109].index
f.drop(indexName,inplace=True)

In [ ]:
# 분기 자료에 단위 맞춰서 더하고 저장하기
with open('Raw_Finance/finance_naver_converted_202103.pickle', 'rb') as handle:
    naver1 = pickle.load(handle)
for col in naver1.columns[2:]:
    naver1[col] = naver1[col]*100000000

naver2 = case.finance_naver
for col in naver2.columns[2:]:
    naver2[col] = naver2[col]*100000000
    
finance = pd.concat([case.finance,naver1])
finance = pd.concat([finance,naver2])
finance.reset_index(drop=True,inplace=True)

with open('Raw_Finance/finance_latest.pickle', 'wb') as handle:
    pickle.dump(finance, handle, protocol=pickle.HIGHEST_PROTOCOL)

# finance에 시총, 성과 저장하기

In [ ]:
def periodToDateBuy(period):
    year = str(period)[:4]
    month = str(period)[4:]
    yearBuy = str(int(year)+1) if month == '12' else year
    monthBuy = '4' if month == '12' else str(int(month)+3)
    dayBuy = '15' if month == '12' else 1
    dateBuy = datetime(int(yearBuy),int(monthBuy),int(dayBuy))
    return dateBuy

df = case.finance

#  기간 별 시가총액 정보를 dict형태로 저장
dicCap = {}
for period in list(df['period']):
    try:
        if period in dicCap.keys():
            continue
        
        dateBuy = periodToDateBuy(period)

        df_cap_kospi = stock.get_market_cap_by_ticker(dateBuy, market='KOSPI')
        df_cap_kosdaq = stock.get_market_cap_by_ticker(dateBuy, market='KOSDAQ')
        df_cap = pd.concat([df_cap_kospi, df_cap_kosdaq], axis=0)
        df_cap = case.cap_add_preferred(df_cap)
        dicCap[period] = df_cap
    except:
        print(period)

#  finance에 넣을 수 있도록 시총 column에 들어갈 값을 리스트 형태로 저장
listCap = [(dicCap[period].at[stock_code,'시가총액'] if stock_code in dicCap[period].index else None) if (period in dicCap.keys()) else None for stock_code, period in zip(df['stock_code'], df['period'])]
df['cap'] = listCap
import pickle
with open('Raw_Finance/finance_input_quarterly_data_cap.pickle','wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

#  finance에 넣을 수 있도록 RAR column에 들어갈 값을 리스트 형태로 저장    
listRAR = []
for stock_code, period in zip(df['stock_code'], df['period']):
    if period in dicCap.keys():
        if stock_code in dicCap[period].index:
            dateStart = periodToDateBuy(period)
            dateEnd = periodToDateBuy(period)+relativedelta(months=3)
            changeTemp = case.change.loc[dateStart:dateEnd,stock_code]
            checkNan = changeTemp.isnull().sum()
            if checkNan != 0:
                listRAR.append(None)
                continue
#             print(type(changeTemp))
#             print(changeTemp)
#             print(checkNan)
            
            rarTemp = case.calculateRegressedReturn(changeTemp)
            listRAR.append(rarTemp)
        else:
            listRAR.append(None)
    else:
        listRAR.append(None)

# listRAR = [ ( case.calculateRegressedReturn(case.change.loc[periodToDateBuy(period):periodToDateBuy(period)+relativedelta(months=3),stock_code]) if stock_code in dicCap[period].index else None) if period in dicCap.keys() else None for stock_code, period in zip(df['stock_code'], df['period'])]

df['rar'] = listRAR
import pickle
with open('Raw_Finance/finance_input_quarterly_data_cap_rar.pickle','wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 3 차원 그래프, 히트맵으로 성과 확인하기

In [ ]:
AxisSell = []
AxisBuy = []
AxisCAGR = []
AxisMDD = []
AxisMAR = []
AxisSharpe = []
AxisnoOfTrade = []
case.daysContinuous = 5
for buy in range(10,61,5):
    for sell in range(10,61,5):
        test1 = case.applyTraidingStrategy(strategy="MAKosdaqContinuous",daysForMABuy=buy,daysForMASell=sell)
        AxisSell.append(sell)
        AxisBuy.append(buy)
        AxisnoOfTrade.append(case.noOfTrade)
        result = case.simple_analyzer(test1)
        AxisCAGR.append(result.at['CAGR','BuyMA'])
        AxisMDD.append(result.at['MDD','BuyMA']*(-1))
        AxisSharpe.append(result.at['Sharpe_Ratio','BuyMA'])
        AxisMAR.append(result.at['CAGR','BuyMA'] / result.at['MDD','BuyMA']*(-1))

In [ ]:
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(AxisBuy, AxisSell, AxisMAR, marker='o', s=15, cmap='Greens')
plt.xlabel('Buy')
plt.ylabel('Sell')
plt.show()

In [ ]:
loss = 0.5  # 잦은 거래로 인한 손실
AxisCAGRadjusted = [a*((1-loss/100)**(b/8.8+4)) for a,b in zip(AxisCAGR, AxisnoOfTrade)]
df = pd.DataFrame({'buy':AxisBuy,'sell':AxisSell,'CAGR':AxisCAGR,'CAGRadj':AxisCAGRadjusted,'MDD':AxisMDD,'sharpe_ratio':AxisSharpe,
                   'MAR':AxisMAR,'noOfTrade':AxisnoOfTrade})

In [ ]:
df = pd.DataFrame({'buy':AxisBuy,'sell':AxisSell,'CAGR':AxisCAGR,'MDD':AxisMDD,'sharpe_ratio':AxisSharpe,
                   'MAR':AxisMAR,'noOfTrade':AxisnoOfTrade})
import seaborn as sns
import matplotlib.pyplot as plt
for i in df.columns[2:]:
    plt.figure(figsize=(7,7))
    plt.title(i)
    if i != 'noOfTrade':
        sns.heatmap(data = df.pivot_table(index=df.columns[0],columns=df.columns[1],values=i), annot=True,fmt = '.2f', linewidths=.2, cmap='Blues')
    else:
        sns.heatmap(data = df.pivot_table(index=df.columns[0],columns=df.columns[1],values=i), annot=True,fmt = '.0f', linewidths=.2, cmap='Blues')
    plt.savefig(f'image/{i}.png')

# 거래빈도 검토

In [ ]:
listCount = []
count = 1
listSignal = list(test['MASignal'])
for i in range(len(listSignal)-1):
    if listSignal[i] == listSignal[i+1]:
        count += 1
    else:
        listCount.append(count)
        count = 1
        
import matplotlib.pyplot as plt
D = dict(Counter(listCount).most_common())
plt.bar(list(D.keys()), list(D.values()),align='edge')

plt.plot(listCount)